Простое удаление и восстановление переносов строки (карта по символам)

In [ ]:
def get_linebreak_map(text: str):
    """Возвращает список позиций переносов строк"""
    return [i for i, ch in enumerate(text) if ch == "\n"]

def replacelinebreaks_with_spaces(text: str):
    """Заменяет \n на пробелы и возвращает карту"""
    lb_map = get_linebreak_map(text)
    return text.replace("\n", " "), lb_map

def restorelinebreaks(text: str, lb_map: list[int]):
    """Восстанавливает переносы по карте"""
    chars = list(text)
    for pos in lb_map:
        chars[pos] = "\n"
    return "".join(chars)


# пример
orig = "раз два\nтри четыре\nпять"
print("ORIG:", repr(orig))

# заменили \n на пробел
prepared, lb_map = replacelinebreaks_with_spaces(orig)
print("PREP:", repr(prepared))
print("MAP:", lb_map)

# ничего не делаем, просто восстанавливаем
restored = restorelinebreaks(prepared, lb_map)
print("RESTORED:", repr(restored))


!!! Возвращение переноса строки после обработки текста (карта по символам)   проверить

In [ ]:
import re
from difflib import SequenceMatcher
from typing import List, Callable

def get_linebreak_map(text: str) -> List[int]:
    return [i for i, ch in enumerate(text) if ch == "\n"]

def replacelinebreaks_with_spaces(text: str):
    lb_map = get_linebreak_map(text)
    # заменяем CRLF/CR на \n заранее, если нужно:
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    prepared = text.replace('\n', ' ')
    return prepared, lb_map

def map_orig_positions_to_processed(orig: str, proc: str, positions: List[int]) -> List[int]:
    """
    Для каждой позиции из `positions` (индексы в orig) возвращает индекс
    в proc, где лучше всего вставить/заменить на '\n'.
    """
    sm = SequenceMatcher(None, orig, proc)
    opcodes = sm.get_opcodes()
    positions_sorted = sorted(positions)
    res = {}
    pos_idx = 0

    for tag, i1, i2, j1, j2 in opcodes:
        # обрабатываем все позиции, попавшие в текущую область оригинала [i1,i2)
        while pos_idx < len(positions_sorted) and positions_sorted[pos_idx] < i2:
            pos = positions_sorted[pos_idx]
            if pos < i1:
                # на всякий случай
                mapped = j1
            else:
                if tag == 'equal':
                    mapped = j1 + (pos - i1)
                elif tag == 'replace':
                    # попытка сохранить относительное смещение, но не выходить за границы j1..j2
                    rel = pos - i1
                    if j2 > j1:
                        mapped = j1 + min(rel, (j2 - j1 - 1))
                    else:
                        mapped = j1
                elif tag == 'delete':
                    # оригинал удалён — вставляем в точку j1 (перед следующими символами)
                    mapped = j1
                elif tag == 'insert':
                    # сюда не попадём, потому что i1==i2 при insert
                    mapped = j1
                else:
                    mapped = j1
            res[pos] = mapped
            pos_idx += 1
            if pos_idx >= len(positions_sorted):
                break
        if pos_idx >= len(positions_sorted):
            break

    # если какие-то позиции остались (после конца), ставим в конец
    while pos_idx < len(positions_sorted):
        res[positions_sorted[pos_idx]] = len(proc)
        pos_idx += 1

    return [res[p] for p in positions]  # в исходном порядке

def restorelinebreaks_by_mapped(proc: str, mapped_positions: List[int]) -> str:
    """
    Восстановление: предпочитаем ЗАМЕНИТЬ пробел на '\n' (на mapped index или рядом),
    иначе вставляем '\n' перед символом с индексом mapped.
    """
    chars = list(proc)
    n = len(chars)
    insert_before = [False] * (n + 1)
    replace_at = set()

    # небольшое окно поиска пробела (чтобы корректно заменить, если пробел сдвинулся)
    WINDOW = 3

    for m in mapped_positions:
        if m < 0:
            m = 0
        if m > n:
            m = n

        placed = False
        # 1) попробуем заменить пробел точно на m
        if m < n and chars[m] == ' ':
            replace_at.add(m)
            placed = True
        # 2) попробуем ближайшие позиции (по возрастанию дистанции): m-1, m+1, m-2, m+2...
        if not placed:
            for d in range(1, WINDOW + 1):
                left = m - d
                right = m + d
                if left >= 0 and left < n and chars[left] == ' ':
                    replace_at.add(left)
                    placed = True
                    break
                if right >= 0 and right < n and chars[right] == ' ':
                    replace_at.add(right)
                    placed = True
                    break
        # 3) если не нашли пробел — вставляем перед m
        if not placed:
            insert_before[m] = True

    # собираем результат: учитываем замены и вставки
    out = []
    for i in range(n):
        if insert_before[i]:
            out.append('\n')
        if i in replace_at:
            out.append('\n')
        else:
            out.append(chars[i])
    if insert_before[n]:
        out.append('\n')
    return ''.join(out)

def process_preservelinebreaks(text: str, processor: Callable[[str], str]) -> str:
    """
    processor: функция, принимающая строку и возвращающая обработанную строку
               (например, accentizer.process_all)
    """
    prepared, lb_map = replacelinebreaks_with_spaces(text)
    processed = processor(prepared)
    mapped = map_orig_positions_to_processed(prepared, processed, lb_map)
    restored = restorelinebreaks_by_mapped(processed, mapped)
    return restored

# ------------- пример -------------
if __name__ == "__main__":
    def fake_accentizer(s: str) -> str:
        # тест: вставка '+' внутри слова "жопа"
        return s.replace(" ", "   ")

    orig = "всё то что\nзовётся жопа\nи грусть"
    print("ORIG  :", repr(orig))
    result = process_preservelinebreaks(orig, fake_accentizer)
    print("RESULT:", repr(result))


Простое удаление и восстановление переносов строки (карта по словам)

In [ ]:
import re
from typing import List, Callable

def words_and_linebreak_map(text: str):
    """
    Возвращает (words, lb_map)
    words: список слов (переносы удалены)
    lb_map: список позиций (число слов перед каждым переносом)
    """
    # нормализуем окончания
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    # делаем \n отдельным "токеном" и разделяем только по пробелу
    parts = text.replace('\n', ' \n ').split(' ')
    # сохраняем непустые токены, но оставляем '\n'
    tokens = [p for p in parts if p != '']
    words = []
    lb_map = []
    wcount = 0
    for t in tokens:
        if t == '\n':
            # перенос после wcount слов
            lb_map.append(wcount)
        else:
            words.append(t)
            wcount += 1
    return words, lb_map

def restore_words_withlinebreaks(words: List[str], lb_map: List[int]) -> str:
    """
    Вставляет переносы по карте и соединяет слова в строку.
    (Сжимает последовательности пробелов до одиночных.)
    """
    lb_set = set(lb_map)
    out_tokens = []
    # переносы в начале (если есть lb_map с 0)
    if 0 in lb_set:
        out_tokens.append('\n')
    for i, w in enumerate(words):
        out_tokens.append(w)
        # если после (i+1)-го слова нужно перенос — добавляем
        if (i + 1) in lb_set:
            out_tokens.append('\n')
    # склеиваем
    s = ' '.join(out_tokens)
    # заменим " <newline> " на настоящий перенос
    return s.replace(' \n ', '\n')

def process_text_wordwise(text: str, word_processor: Callable[[str], str]):
    """
    Вход: исходный текст и функция word_processor(word)->processed_word.
    Работает по словам, возвращает текст с восстановленными переносами.
    """
    words, lb_map = words_and_linebreak_map(text)
    # обработка слова за словом (сохраняется 1:1 соответствие)
    processed = [word_processor(w) for w in words]
    return restore_words_withlinebreaks(processed, lb_map)

# ---------- пример использования ----------
if __name__ == "__main__":
    orig = """в иллюминатор     постучали
и изменившийся в лице
гагарин ищет по карманам
кэ цэ
"""
    print("ORIG:  ", repr(orig))

    # эмуляция процессора: ставит ударение в одном слове
    def fake_proc(w):
        return w.replace("лице", "лиц+е")

    out = process_text_wordwise(orig, fake_proc)
    print("RESULT:", repr(out))
    # ожидаем: 'всё то что\nзовётся ж+опа\nи грусть'


Возвращение переноса строки после обработки текста (карта по словам)   показана только карта       полная функция ниже и в linebreaks.py

In [ ]:
def update_linebreak_map(word_map, changes):
    """
    Корректирует карту переносов при изменении числа слов.

    word_map: список индексов переносов (например [4, 7, 10])
    changes: список кортежей (pos, old_count, new_count)
             pos       – индекс слова в исходном тексте (где произошла замена)
             old_count – сколько слов было
             new_count – сколько слов стало
    """
    # сортируем изменения по позиции, чтобы применять по порядку
    changes = sorted(changes, key=lambda x: x[0])
    new_map = word_map[:]
    shift = 0

    for pos, old, new in changes:
        delta = new - old
        # все переносы, идущие ПОСЛЕ или НА этой позиции, двигаются
        new_map = [idx + delta if idx >= pos + shift else idx for idx in new_map]
        shift += delta

    return new_map

word_map = [4, 7, 10]
changes = [(5, 2, 1)]

print(update_linebreak_map(word_map, changes))



!!! Возвращение переноса строки после обработки текста (карта по словам)   проверить       см в linebreaks.py

In [ ]:
import difflib
from collections import Counter
from typing import List, Tuple, Callable

def words_and_map(text: str) -> Tuple[List[str], List[int]]:
    """
    Возвращает (words, word_map).
    word_map содержит позиции (число слов до переноса), например [3, 6].
    Сохраняет переносы как отдельный токен '\n'.
    """
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    # делаем '\n' отдельным токеном, затем split по одиночному пробелу,
    # чтобы не терять маркеры '\n'
    spaced = text.replace('\n', ' \n ')
    parts = spaced.split(' ')  # split(' ') сохраняет пустые элементы для множественных пробелов
    tokens = [p for p in parts if p != '']  # удаляем пустые элементы, но сохраняем '\n'
    words = []
    word_map = []
    wcount = 0
    for t in tokens:
        if t == '\n':
            # перенос после wcount слов (то есть между wcount-1 и wcount)
            word_map.append(wcount)
        else:
            words.append(t)
            wcount += 1
    return words, word_map

def _ensure_list_words(out) -> List[str]:
    """Нормализует результат процессора в список слов."""
    if isinstance(out, list):
        return out
    if isinstance(out, str):
        return out.split()
    # любая итерируемая последовательность
    return list(out)

def try_process(before_words: List[str], processor: Callable) -> List[str]:
    """
    Попытка вызвать processor двумя способами:
      1) processor(before_words) — если процессор умеет работать с list
      2) processor(' '.join(before_words)) — если процессор ожидает строку
    Возвращает список слов (после .split()).
    """
    try:
        out = processor(before_words)
    except TypeError:
        out = processor(' '.join(before_words))
    return _ensure_list_words(out)

def map_boundaries(before: List[str], after: List[str], positions: List[int]) -> List[int]:
    """
    Для каждого boundary в positions (0..len(before)) возвращает позицию в after,
    означающую сколько слов идёт перед соответствующим переносом в обработанном тексте.
    Использует opcodes SequenceMatcher для корректного сопоставления блоков.
    """
    sm = difflib.SequenceMatcher(a=before, b=after)
    opcodes = sm.get_opcodes()
    mapped = []
    for pos in positions:
        m = 0
        for tag, i1, i2, j1, j2 in opcodes:
            if pos >= i2:
                # целиком лежит до границы — добавляем весь блок after-length
                m += (j2 - j1)
                continue
            if pos <= i1:
                # граница лежит до начала этого блока — больше ничего не добавляем
                break
            # граница внутри текущего блока: i1 < pos < i2
            # если блок равный, можно точно добавить соответствующую часть
            if tag == 'equal':
                m += (pos - i1)
            else:
                # если это replace/delete/insert — разумный выбор:
                # считаем, что граница映ируется в начало соответствующего блока after (j1)
                # (альтернативы: j1 + min(pos-i1, j2-j1) — но это делает поведение менее детерминированным)
                m += 0
            break
        mapped.append(m)
    return mapped

#  ???  а если сделать универсальный вход - (list или str) 
def restorelinebreaks_from_words(words: List[str], word_map: List[int]) -> str:
    """
    Склеивает список слов в строку, вставляя '\n' согласно word_map (число слов до переноса).
    Поддерживает несколько переносов подряд (повторы позиции).
    """
    counts = Counter(word_map)
    lines = []
    # переносы в начале (если есть позиции 0)
    for _ in range(counts.get(0, 0)):
        lines.append('')
    current = []
    for i, w in enumerate(words, start=1):
        current.append(w)
        c = counts.get(i, 0)
        if c:
            lines.append(' '.join(current))
            for _ in range(c - 1):
                lines.append('')  # пустые строки при последовательных переносах
            current = []
    if current:
        lines.append(' '.join(current))
    return '\n'.join(lines)

def linebreaks_restore(text: str, processor: Callable) -> Tuple[str, dict]:
    """
    Универсальная обёртка:
      - строит карту переносов по словам,
      - пропускает через processor (list->list или str->str),
      - строит изменения / маппинг границ и восстанавливает '\n'.
    Возвращает (restored_text, debug_dict)
    """
    before_words, old_map = words_and_map(text)
    after_words = try_process(before_words, processor)
    new_map = map_boundaries(before_words, after_words, old_map)
    restored = restorelinebreaks_from_words(after_words, new_map)
    debug = {
        'before_words': before_words,
        'after_words': after_words,
        'old_map': old_map,
        'new_map': new_map,
    }
    return restored, debug

# ========== пример ==========

if __name__ == "__main__":
    text = "всё то что\n125 зовётся жопа\nи грусть"

    def fake_numberizer(tokens):
        out = []
        for t in tokens:
            if t == "125":
                out.extend(["сто", "двадцать", "пять"])
            else:
                out.append(t)
        return out

    restored, dbg = linebreaks_restore(text, fake_numberizer)
    print("ORIG:", repr(text))
    print("BEFORE:", dbg['before_words'])
    print("OLD_MAP:", dbg['old_map'])
    print("AFTER:", dbg['after_words'])
    print("NEW_MAP:", dbg['new_map'])
    print("RESTORED:\n" + restored)


===========================================================

пунктуация (открывающая и закрывающая) корректное возвращение мз списка в строку

In [2]:
from typing import List

CLOSE_PUNCT = ".,!?;:)]}»"
OPEN_PUNCT  = "([{\"«"

def detokenize(tokens: List[str]) -> str:
    out = []
    for i, tok in enumerate(tokens):
        if i == 0:
            out.append(tok)
            continue

        prev = tokens[i-1]

        if tok in CLOSE_PUNCT:
            # слева без пробела
            out[-1] = out[-1] + tok
        elif prev in OPEN_PUNCT:
            # справа без пробела
            out[-1] = out[-1] + tok
        else:
            # обычный случай
            out.append(" " + tok)
    return "".join(out)


# пример
print(detokenize(["(", "привет", ",", "друг", "!", ")", "ля дя", "!"]))
# -> "(привет, друг)"


(привет, друг!) ля дя!
